In [1]:
"""
XOR gate by Tensorflow
"""
import numpy as np
train = np.array([[0,0],[0,1],[1,0],[1,1]])
label = np.array([[0],[1],[1],[0]])

In [2]:
import tensorflow as tf
class MLP(tf.keras.Model): #MultiLayerPerceptron
    """
    l1(Dense);隠れ層
    l2(Dense):出力層
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.l1 = tf.keras.layers.Dense(
            units=hidden_dim,
            input_dim=input_dim,
            activation="sigmoid",
        )
        self.l2 = tf.keras.layers.Dense(
            units = output_dim,
            activation="sigmoid",
        )
    
    @tf.function
    def call(self, x, training = None):
        """
        Function of callback from instance of model

        ”forward propagation processing（順伝播による処理）”

        x(ndarray(float32)): train data
        training(bool): train True, varidation False
        Return : output
        """
        h = self.l1(x)
        y = self.l2(h)
        return y

model = MLP(2,2,1)



In [3]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.5)

In [4]:
"""
Update Parameter
"""
@tf.function
def train_step(x, t):
    """
    x: train data
    t: label
    return: pred_loss
    """
    with tf.GradientTape() as tape:
        predictions = model(x)
        pred_loss = loss_fn(t, predictions)
    
    gradients = tape.gradient(
        pred_loss,
        model.trainable_variables,
    )
    
    optimizer.apply_gradients(
        zip(gradients, model.trainable_variables)
    )
    return pred_loss


In [5]:
"""
Learning
"""
epochs = 4000

for epoch in range(epochs):
    epoch_loss = 0.

    loss = train_step(train, label)
    epoch_loss += loss.numpy()

    if (epoch + 1)%1000 ==0:
        print("epoch({}) loss: {: 4f}".format(epoch+1, epoch_loss))

model.summary()

2022-06-06 12:43:09.672665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-06 12:43:09.673751: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


epoch(1000) loss:  0.399343
epoch(2000) loss:  0.358488
epoch(3000) loss:  0.352922
epoch(4000) loss:  0.350858
Model: "mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  6         
_________________________________________________________________
dense_1 (Dense)              multiple                  3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [6]:
"""
Evaluate model
"""
print(model(train))
print(tf.cast(((model(train)) >= 0.5) , tf.int32))

tf.Tensor(
[[0.00538131]
 [0.9949844 ]
 [0.4991716 ]
 [0.5025197 ]], shape=(4, 1), dtype=float32)
tf.Tensor(
[[0]
 [1]
 [0]
 [1]], shape=(4, 1), dtype=int32)
